In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

torch.manual_seed(1337)
B,T,C = 4, 8, 2 # Batch size, sequence length, and number of channels
x = torch.randn(B, T, C)

In [2]:
torch.no_grad()

In [3]:
torch.manual_seed(42)

# the mat mul operation
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

# rows times columns
c = a @ b

print (a)
print (b)
print (c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [4]:
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)

xbow2 = weights @ x # (T, T) @ (B, T, C) -> (B, T, C)

In [5]:
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

## timestamp

https://youtu.be/kCc8FmEb1nY?si=pq2FmeBHG37mZymE&t=3282


In [8]:
torch.tril(torch.ones(T, T))

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
import torch.nn.functional as F

# version 3: uses softmax for normalization
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros(T, T)

# with the next operation we set the upper triangular part to -inf
weights = weights.masked_fill(tril == 0, float('-inf'))  # set upper triangular to -inf
weights = F.softmax(weights, dim=-1)  # apply softmax to normalize

#the -inf is to ensure tokens from the past dont comunicati with tokens from the future
xbow3 = weights @ x  # (T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow2, xbow3)  # should be True, both methods yield the same result

True

In [18]:
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [34]:
# self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch size, time steps, channels
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2, -1) # (B,T,16) * (B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T, T)) # (T,T) # don't consider future tokens
wei = wei.masked_fill(tril == 0, float('-inf')) # (B,T,T) # set 0 to -inf for softmax
wei = F.softmax(wei, dim=-1) # (B,T,T) # apply softmax to sum to 1

v = value(x)    # (B,T,16)
output = wei @ v # (B,T,T) @ (B,T,16) -> (B,T,16) # attention output for each token in the sequence

In [ ]:
# Andrej Karpathy explanation
# for a token, my information is kept in vector x
# query, here is what I'm interested in
# key, here is what I have
# If you find me interesting, here is what I will communicate.

# by default, these nodes (tokens) have no way of knowing where the are in the sequence
# we can add positional encodings to the token embeddings to give them a sense of order

# for encoder attentionk, the masked_prefil is not needed, because all tokens can attend to each other


# self attention from vs cross-attention
# In this case, it is self attention, because the output of the attention mechanism moves the embedding vector of the token itself based on interaction with the other tokens in the sequence
# k, q, v are all derived from the same source, the token embeddings of the sequence itself

# in cross-attention, the k and v are derived from a different source than the q
# for example, in a language model, the k and v are derived from the token embeddings of the sequence itself, but the q is derived from the token embeddings of the sequence itself

In [ ]:
# from attention is all you need paper (2017)

# we only need to add the division by sqrt(head_size) when calculating the attention weights,

# why do we do this:
# to prevent the dot products from growing too large in magnitude
# its called "scaled dot-product attention"

k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) # * head_size**-0.5 # (B,T,16) * (B,16,T) -> (B,T,T)

print(k.var(), q.var(), wei.var())
# variance of wei is in the order of head_size

# but if we scale it down by sqrt(head_size)
wei = wei * head_size**-0.5
print(wei.var())

# wei feeds into softmax, and softmax is sensitive to large input values
# this means softmax results in one-hot like vectors, which is not what we want

# normally distributed inputs to softmax
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

# sharpen towards the max
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)

# so we scale it down by sqrt(head_size) to prevent the dot products from growing too large in magnitude

tensor(1.0010) tensor(1.0562) tensor(17.5985)
tensor(1.0999)


tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])